In [1]:
#!/usr/bin/env python3
import argparse
from importlib import import_module
from itertools import count
import os

import h5py
import json
import numpy as np
from sklearn.metrics import average_precision_score
import tensorflow as tf

import commoncuhk
import loss

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.utils.np_utils import to_categorical
from keras.layers import LeakyReLU, Concatenate, concatenate, Lambda, UpSampling2D, Add, Input, Dense
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, BatchNormalization

In [2]:
class Args:
    excluder= 'market1501' #, 'diagonal','cuhk03','duke'
    query_dataset = '/home/k/kajal/triplet-reid/data/cuhk03-np/detected_query.csv'
    query_embeddings = '/home/k/kajal/triplet-reid/experiments/cuhk_pepnet/query_embeddings.h5'
    query_embeddings_adv = '/home/k/kajal/triplet-reid/experiments/cuhk1rec/query_embeddings.h5'
    gallery_dataset = '/home/k/kajal/triplet-reid/data/cuhk03-np/detected_test.csv'
    gallery_embeddings =  '/home/k/kajal/triplet-reid/experiments/cuhk_pepnet/gallery_embeddings.h5'
    metric =  'euclidean'
    #filename = 
    batch_size = 18

In [3]:
args=Args()

In [4]:
# Load the query and gallery data from the CSV files.
query_pids, query_fids = commoncuhk.load_dataset(args.query_dataset, None)
gallery_pids, gallery_fids = commoncuhk.load_dataset(args.gallery_dataset, None)

# Load the two datasets fully into memory.
with h5py.File(args.query_embeddings, 'r') as f_query:
    query_embs = np.array(f_query['emb'])
    query_embs_dp = np.array(f_query['emb'])


with h5py.File(args.gallery_embeddings, 'r') as f_gallery:
    gallery_embs = np.array(f_gallery['emb'])
    gallery_embs_dp = np.array(f_gallery['emb'])

In [5]:
gallery_embs=gallery_embs
gallery_embs_dp=gallery_embs_dp
gallery_pids=gallery_pids

In [6]:
uni_pids=np.unique(gallery_pids)
print(len(uni_pids))
uni_query_pids=np.unique(query_pids)
len(uni_query_pids)

700


700

In [7]:
print(query_embs.shape)
print(gallery_embs.shape)

(1400, 128)
(5332, 128)


In [8]:
print(query_embs[0],query_embs[1])

[ 2.2034714  -1.2927732   0.87319475  0.708815    3.128399    0.10258525
  0.5819795  -5.3290186  -0.00696615 -0.65817046  4.1388006  -0.1405488
  3.6700864  -5.9736433  -0.18021151 -1.9956342   5.06066     0.5869995
  2.2143734  -1.500974    1.9035482  -1.7543072  -0.6153123   2.0689025
 -2.5458465  -1.4771395  -0.44672918  2.3772175  -1.2336417  -0.10243905
 -4.1896257   3.1497483  -1.0488262   2.6556268   1.0072933   1.9376538
  0.635422   -4.052031    0.27907348  2.953896   -0.54858744  2.072812
 -3.903998   -1.0724238   1.0174351   0.34009045  0.7049166  -0.516996
  0.49609482 -4.4458303   0.44458765  1.0506483  -0.6284567   2.8564775
  1.3777007  -0.8946021   2.0027823  -1.362541    0.320893    1.0560054
  0.94164836 -4.468156    2.9926534   1.387099    1.973374    5.056146
  0.5344552  -1.3215115  -3.3295658  -0.18862541  0.50694364 -0.12669206
 -0.24016316  4.1486664   0.35141796 -1.4002954  -1.8437268   0.05918837
 -1.4106699   0.51893854  1.6829897   3.209666    0.75877595  2

In [9]:
#query_embs_dp=query_embs
query_embs_dp.shape
query_embs[0]
print(np.min(query_embs),np.mean(query_embs),np.max(query_embs),np.std(query_embs))
print(np.min(gallery_embs),np.mean(gallery_embs),np.max(gallery_embs),np.std(gallery_embs))

-9.129788 -0.03733886 8.56692 1.9269185
-9.302376 -0.03968162 8.700699 1.9250435


In [10]:
query_embs_dp[0,1]=np.random.normal(0,1.9269185)
query_embs_dp=np.exp(0.1) * query_embs_dp +0.001
gallery_embs_dp[0,1]=np.random.normal(0,1.9269185)
gallery_embs_dp=np.exp(0.1) * gallery_embs_dp +0.001

In [11]:
def l1(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.max(np.subtract(actual,pred)).mean()

def mse(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.max(np.square(np.subtract(actual,pred)).mean())

D0 = query_embs + np.random.normal(0,1.9269185)
D1 = query_embs_dp + np.random.normal(0,1.9269185)

D2 = gallery_embs + np.random.normal(0,1.9269185)
D3 = gallery_embs_dp + np.random.normal(0,1.9269185)
#print(D0,D1)
#sens1 = l1(query_embs,query_embs_dp)
sensq = mse(D0,D1)
sensg = mse(D2,D3)
print(sensq,sensg)

13.701109 18.082117


In [12]:
query_data = query_pids
query_values = array(query_data)
print('values', query_values)
# integer encode
query_label_encoder = LabelEncoder()
query_integer_encoded = query_label_encoder.fit_transform(query_values)
print('encoded', query_integer_encoded)
# binary encode
query_onehot_encoder = OneHotEncoder(sparse=False)
query_integer_encoded = query_integer_encoded.reshape(len(query_integer_encoded), 1)
query_onehot_encoded = query_onehot_encoder.fit_transform(query_integer_encoded)
print(query_onehot_encoded.shape)

values ['722' '103' '156' ... '369' '374' '1107']
encoded [538  18 234 ... 351 354  53]
(1400, 700)


In [13]:
data = gallery_pids
values = array(data)
print('values', values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print('encoded', integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded.shape)

values ['136' '704' '303' ... '1111' '802' '1199']
encoded [177 527 314 ...  56 586 101]
(5332, 700)


In [14]:
X_train  = D3
Y_train_label = onehot_encoded
X_test = D0
Y_test_label = query_onehot_encoded
#print(X_train.size)
#print(X_test.size)
#print(Y_train_label)
#print(Y_test_label)

In [15]:
#print(Y_train_label[2:5000])
#y_train_label = to_categorical(Y_train_label)
#y_train_label.shape

In [16]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
#model.add( Dense(units=512, input_dim=128, kernel_initializer='normal', activation='relu') )
#model.add( Dropout(0.5))
model.add( Dense(units=700, input_dim=128, kernel_initializer='normal', activation='softmax') )
print( model.summary() )
model.compile( loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 700)               90300     
                                                                 
Total params: 90,300
Trainable params: 90,300
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
train_history = model.fit(x=X_train, y=Y_train_label, validation_split=0.2, epochs=6, batch_size=128, verbose=2)

Epoch 1/6
34/34 - 3s - loss: 6.4131 - accuracy: 0.0265 - val_loss: 5.7672 - val_accuracy: 0.0366 - 3s/epoch - 100ms/step
Epoch 2/6
34/34 - 1s - loss: 4.3512 - accuracy: 0.2326 - val_loss: 4.6371 - val_accuracy: 0.1612 - 522ms/epoch - 15ms/step
Epoch 3/6
34/34 - 0s - loss: 3.0645 - accuracy: 0.5329 - val_loss: 3.8162 - val_accuracy: 0.2952 - 342ms/epoch - 10ms/step
Epoch 4/6
34/34 - 0s - loss: 2.2169 - accuracy: 0.7503 - val_loss: 3.2424 - val_accuracy: 0.3861 - 285ms/epoch - 8ms/step
Epoch 5/6
34/34 - 0s - loss: 1.6468 - accuracy: 0.8530 - val_loss: 2.8111 - val_accuracy: 0.4789 - 300ms/epoch - 9ms/step
Epoch 6/6
34/34 - 0s - loss: 1.2675 - accuracy: 0.9069 - val_loss: 2.5051 - val_accuracy: 0.5351 - 322ms/epoch - 9ms/step


In [18]:
print("Evaluate on test data")
results = model.evaluate(X_test, Y_test_label)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions)

Evaluate on test data
44/44 [==============================] - 0s 5ms/step - loss: 2.5581 - accuracy: 0.4114
test loss, test acc: [2.5580813884735107, 0.4114285707473755]
Generate predictions for 3 samples
44/44 [==============================] - 0s 4ms/step
predictions shape: [[2.20238324e-03 5.06063807e-04 1.63405514e-06 ... 2.70761757e-05
  1.35950741e-05 3.45673106e-05]
 [3.58659927e-05 9.97593510e-04 1.00829020e-05 ... 1.49146272e-05
  8.03363582e-05 3.83569393e-04]
 [1.38842268e-04 5.05935343e-04 4.05818264e-06 ... 1.54798181e-04
  1.05616709e-05 5.79839179e-05]
 ...
 [7.27685574e-06 7.42731027e-06 1.91160848e-06 ... 7.87551107e-05
  1.62542819e-05 8.70798540e-05]
 [5.82680186e-05 1.98728614e-03 2.20124434e-06 ... 1.13364724e-04
  6.79065415e-05 2.55317300e-05]
 [1.50195331e-06 2.15825230e-01 1.42132951e-04 ... 2.72897450e-04
  1.02202685e-05 5.93600911e-04]]


In [ ]:
from diffprivlib.models import GaussianNB
clf = GaussianNB(epsilon=1,bounds=(-50e50, 50e50))
clf.fit(X_train, gallery_pids)

In [ ]:
clf.predict(X_test)
print("Test accuracy: %f" % clf.score(X_test, query_pids))

In [ ]:
def Hbeta(D=np.array([]), beta=1.0):
    """
        Compute the perplexity and the P-row for a specific value of the
        precision of a Gaussian distribution.
    """

    # Compute P-row and corresponding perplexity
    P = np.exp(-D.copy() * beta)
    sumP = sum(P)
    H = np.log(sumP) + beta * np.sum(D * P) / sumP
    P = P / sumP
    return H, P

def x2p(X=np.array([]), tol=1e-5, perplexity=30.0):
    """
        Performs a binary search to get P-values in such a way that each
        conditional Gaussian has the same perplexity.
    """

    # Initialize some variables
    print("Computing pairwise distances...")
    (n, d) = X.shape
    sum_X = np.sum(np.square(X), 1)
    D = np.add(np.add(-2 * np.dot(X, X.T), sum_X).T, sum_X)
    P = np.zeros((n, n))
    beta = np.ones((n, 1))
    logU = np.log(perplexity)

    # Loop over all datapoints
    for i in range(n):

        # Print progress
        if i % 500 == 0:
            print("Computing P-values for point %d of %d..." % (i, n))

        # Compute the Gaussian kernel and entropy for the current precision
        betamin = -np.inf
        betamax = np.inf
        Di = D[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))]
        (H, thisP) = Hbeta(Di, beta[i])

        # Evaluate whether the perplexity is within tolerance
        Hdiff = H - logU
        tries = 0
        while np.abs(Hdiff) > tol and tries < 50:

            # If not, increase or decrease precision
            if Hdiff > 0:
                betamin = beta[i].copy()
                if betamax == np.inf or betamax == -np.inf:
                    beta[i] = beta[i] * 2.
                else:
                    beta[i] = (beta[i] + betamax) / 2.
            else:
                betamax = beta[i].copy()
                if betamin == np.inf or betamin == -np.inf:
                    beta[i] = beta[i] / 2.
                else:
                    beta[i] = (beta[i] + betamin) / 2.

            # Recompute the values
            (H, thisP) = Hbeta(Di, beta[i])
            Hdiff = H - logU
            tries += 1

        # Set the final row of P
        P[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))] = thisP

    # Return final P-matrix
    print("Mean value of sigma: %f" % np.mean(np.sqrt(1 / beta)))
    return P

def pca(X=np.array([]), no_dims=50):
    """
        Runs PCA on the NxD array X in order to reduce its dimensionality to
        no_dims dimensions.
    """

    print("Preprocessing the data using PCA...")
    (n, d) = X.shape
    X = X - np.tile(np.mean(X, 0), (n, 1))
    (l, M) = np.linalg.eig(np.dot(X.T, X))
    Y = np.dot(X, M[:, 0:no_dims])
    return Y
def tsne(X=np.array([]), no_dims=2, initial_dims=50, perplexity=30.0):
    """
        Runs t-SNE on the dataset in the NxD array X to reduce its
        dimensionality to no_dims dimensions. The syntaxis of the function is
        `Y = tsne.tsne(X, no_dims, perplexity), where X is an NxD NumPy array.
    """

    # Check inputs
    if isinstance(no_dims, float):
        print("Error: array X should have type float.")
        return -1
    if round(no_dims) != no_dims:
        print("Error: number of dimensions should be an integer.")
        return -1

    # Initialize variables
    X = pca(X, initial_dims).real
    (n, d) = X.shape
    max_iter = 10
    initial_momentum = 0.5
    final_momentum = 0.8
    eta = 200
    min_gain = 0.01
    Y = np.random.randn(n, no_dims)
    dY = np.zeros((n, no_dims))
    iY = np.zeros((n, no_dims))
    gains = np.ones((n, no_dims))

    # Compute P-values
    P = x2p(X, 1e-5, perplexity)
    P = P + np.transpose(P)
    P = P / np.sum(P)
    P = P * 4.									# early exaggeration
    P = np.maximum(P, 1e-12)
    

    # Run iterations
    for iter in range(max_iter):

        # Compute pairwise affinities
        sum_Y = np.sum(np.square(Y), 1)
        num = -2. * np.dot(Y, Y.T)
        num = 1. / (1. + np.add(np.add(num, sum_Y).T, sum_Y))
        num[range(n), range(n)] = 0.
        Q = num / np.sum(num)
        Q = np.maximum(Q, 1e-12)

        # Compute gradient
        PQ = P - Q
        for i in range(n):
            dY[i, :] = np.sum(np.tile(PQ[:, i] * num[:, i], (no_dims, 1)).T * (Y[i, :] - Y), 0)

        # Perform the update
        if iter < 20:
            momentum = initial_momentum
        else:
            momentum = final_momentum
        gains = (gains + 0.2) * ((dY > 0.) != (iY > 0.)) + \
                (gains * 0.8) * ((dY > 0.) == (iY > 0.))
        gains[gains < min_gain] = min_gain
        iY = momentum * iY - eta * (gains * dY)
        Y = Y + iY
        Y = Y - np.tile(np.mean(Y, 0), (n, 1))

        # Compute current value of cost function
        if (iter + 1) % 10 == 0:
            C = np.sum(P * np.log(P / Q))
            print("Iteration %d: error is %f" % (iter + 1, C))

        # Stop lying about P-values
        if iter == 100:
            P = P / 4.

    # Return solution
    return Y
             

In [ ]:
import pylab
X = gallery_embs[500:700]
lbl=list(map(int,gallery_pids))
labels = lbl[500:700]#query_pids[0:20]
pylab.figure()
Y = tsne(X, 2, 50, 20.0)
plt=pylab.scatter(Y[:, 0],Y[:,1],10,labels,marker='o')
pylab.axis('off')
pylab.savefig('rsltn3n1.pdf')
pylab.show()

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_test, Y_test_label)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions)

In [ ]:
import pylab
X = query_embs[500:700]
lbl=list(map(int,query_pids))
labels = lbl[500:700]#query_pids[0:20]
pylab.figure()
Y = tsne(X, 2, 50, 20.0)
plt=pylab.scatter(Y[:, 0],Y[:,1],10,labels,marker='o')
pylab.axis('off')
pylab.savefig('rsltn3n2.pdf')
pylab.show()

In [ ]:
    #file = query_embs
    #filel = query_pids
    X = query_embs[0:50]
    lbl=list(map(int,query_pids))
    labels = lbl[0:50]#query_pids[0:20]
    print(query_embs,query_pids)
    #X = np.loadtxt("mnist2500_X.txt")
    pylab.figure()
    Y = tsne(X, 2, 50, 20.0)
    plt=pylab.scatter(Y[:, 0],Y[:,1],30,labels,marker='+')
    pylab.axis('off')
    pylab.legend('on')
    pylab.savefig('rsltn3n1.pdf')
    pylab.show()

In [ ]:
    #file = query_embs
    #filel = query_pids
    X = query_embs[0:50]
    lbl=list(map(int,query_pids))
    labels = lbl[0:50]#query_pids[0:20]
    print(query_embs,query_pids)
    #X = np.loadtxt("mnist2500_X.txt")
    pylab.figure()
    Y = tsne(X, 2, 50, 20.0)
    plt=pylab.scatter(Y[:, 0],Y[:,1],30,labels,marker='+')
    pylab.axis('off')
    pylab.legend('on')
    pylab.savefig('rsltn3n1.pdf')
    pylab.show()